## Efficient Yelp API Calls

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [3]:
# Load API Credentials
with open('/Users/milkasantana/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)



In [13]:
# set our API call parameters 
LOCATION = 'Los angeles, California'
TERM = 'Mexican'


In [14]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_CA_mexican.json"
JSON_FILE


'Data/results_in_progress_CA_mexican.json'

In [15]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_CA_mexican.json not found. Saving empty list to file.


In [16]:
## Determine how many results are already in the file

## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [17]:
# use our yelp_api variable's search_query method to perform our API call
    
results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [18]:
## How many results total?
total_results = results['total']
total_results


12800

In [19]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [20]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

640

In [21]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [22]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/640 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [24]:
# Convert .json to dataframe

# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,MjWMuWJNlxNXr-KmAXL8KQ,madre-oaxacan-restaurant-and-mezcaleria-los-an...,Madre Oaxacan Restaurant and Mezcaleria,https://s3-media1.fl.yelpcdn.com/bphoto/ah5yPI...,False,https://www.yelp.com/biz/madre-oaxacan-restaur...,1202,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.0287002, 'longitude': -118.411...","[pickup, delivery]",$$,"{'address1': '10426 National Blvd', 'address2'...",+13105594732,(310) 559-4732,961.353814
1,VqxDHnbjfwwJXON7ERabOw,hermanito-los-angeles,Hermanito,https://s3-media1.fl.yelpcdn.com/bphoto/5vyBgJ...,False,https://www.yelp.com/biz/hermanito-los-angeles...,614,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 34.04048617, 'longitude': -118.44...","[pickup, delivery]",$$$,"{'address1': '2024 Sawtelle Blvd', 'address2':...",+14243882585,(424) 388-2585,3733.554572
2,Ia6VPQAS8VHdlujcEQ1JWQ,hijo-de-su-madre-los-angeles,Hijo De Su Madre,https://s3-media4.fl.yelpcdn.com/bphoto/9UQmvU...,False,https://www.yelp.com/biz/hijo-de-su-madre-los-...,127,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",5.0,"{'latitude': 34.04433900491594, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '2180 Westwood Blvd', 'address2':...",+14243854672,(424) 385-4672,3317.651207
3,fTgq9ksRbi1hSPkYREoGDA,sol-agave-santa-monica-santa-monica-8,Sol Agave - Santa Monica,https://s3-media1.fl.yelpcdn.com/bphoto/cjzqsd...,False,https://www.yelp.com/biz/sol-agave-santa-monic...,473,"[{'alias': 'newmexican', 'title': 'New Mexican...",4.5,"{'latitude': 34.00923, 'longitude': -118.49173}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '1733 Ocean Ave', 'address2': 'St...",+14242388076,(424) 238-8076,7573.472066
4,LXhSp1RYSHIEZ4c01ewmPA,loqui-culver-city,Loqui,https://s3-media2.fl.yelpcdn.com/bphoto/wjV3vE...,False,https://www.yelp.com/biz/loqui-culver-city?adj...,807,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.0274352, 'longitude': -118.387...",[delivery],$$,"{'address1': '8830 Washington Blvd', 'address2...",+19499972800,(949) 997-2800,2268.934966


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,4zU-v2OxDSosvNpFb37whw,the-court-cafe-los-angeles,The Court Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/1wWpEl...,False,https://www.yelp.com/biz/the-court-cafe-los-an...,552,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 33.9767818448418, 'longitude': -1...","[restaurant_reservation, delivery]",$$,"{'address1': '5496 W Centinela Ave', 'address2...",+13104314969,(310) 431-4969,5759.399143
996,qSjl8vLrZQhWDc2ofXoBwQ,golazo-restaurant-los-angeles,Golazo Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/2GLReA...,False,https://www.yelp.com/biz/golazo-restaurant-los...,9,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.0112549, 'longitude': -118.277...",[delivery],$,{'address1': '245 W Martin Luther King Jr Blvd...,+13232350102,(323) 235-0102,12305.581654
997,w0cm7DFmBDQ_x2r83cvL6g,apollonias-pizzeria-los-angeles-2,Apollonia's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/7y6Mbz...,False,https://www.yelp.com/biz/apollonias-pizzeria-l...,814,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 34.06188, 'longitude': -118.34278}",[delivery],$$,"{'address1': '5176 Wilshire Blvd', 'address2':...",+13239372823,(323) 937-2823,7787.541613
998,W35Y45Jue534WnB9NzDWEg,gutierrez-mexican-restaurant-moorpark-2,Gutierrez Mexican Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/5f6EoB...,False,https://www.yelp.com/biz/gutierrez-mexican-res...,132,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.2784839254345, 'longitude': -1...",[delivery],$,"{'address1': '304 W Los Angeles Ave', 'address...",+18055297862,(805) 529-7862,52328.241849
999,Wr1NPE3LT7Yly79gn4KlSQ,panxa-cocina-long-beach,Panxa Cocina,https://s3-media2.fl.yelpcdn.com/bphoto/JJ_bX1...,False,https://www.yelp.com/biz/panxa-cocina-long-bea...,1739,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 33.764576, 'longitude': -118.1465...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '3937 E Broadway', 'address2': ''...",+15624337999,(562) 433-7999,37457.446533


In [26]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()


252

In [27]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [29]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_CA_mexican.csv.gz', compression='gzip',index=False)

## FEEDBACK: Turn results into a df and view the head, tail and info() of the dataframe

In [31]:
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,MjWMuWJNlxNXr-KmAXL8KQ,madre-oaxacan-restaurant-and-mezcaleria-los-an...,Madre Oaxacan Restaurant and Mezcaleria,https://s3-media1.fl.yelpcdn.com/bphoto/ah5yPI...,False,https://www.yelp.com/biz/madre-oaxacan-restaur...,1202,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.0287002, 'longitude': -118.411...","[pickup, delivery]",$$,"{'address1': '10426 National Blvd', 'address2'...",+13105594732,(310) 559-4732,961.353814
1,VqxDHnbjfwwJXON7ERabOw,hermanito-los-angeles,Hermanito,https://s3-media1.fl.yelpcdn.com/bphoto/5vyBgJ...,False,https://www.yelp.com/biz/hermanito-los-angeles...,614,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 34.04048617, 'longitude': -118.44...","[pickup, delivery]",$$$,"{'address1': '2024 Sawtelle Blvd', 'address2':...",+14243882585,(424) 388-2585,3733.554572
2,Ia6VPQAS8VHdlujcEQ1JWQ,hijo-de-su-madre-los-angeles,Hijo De Su Madre,https://s3-media4.fl.yelpcdn.com/bphoto/9UQmvU...,False,https://www.yelp.com/biz/hijo-de-su-madre-los-...,127,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",5.0,"{'latitude': 34.04433900491594, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '2180 Westwood Blvd', 'address2':...",+14243854672,(424) 385-4672,3317.651207
3,fTgq9ksRbi1hSPkYREoGDA,sol-agave-santa-monica-santa-monica-8,Sol Agave - Santa Monica,https://s3-media1.fl.yelpcdn.com/bphoto/cjzqsd...,False,https://www.yelp.com/biz/sol-agave-santa-monic...,473,"[{'alias': 'newmexican', 'title': 'New Mexican...",4.5,"{'latitude': 34.00923, 'longitude': -118.49173}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '1733 Ocean Ave', 'address2': 'St...",+14242388076,(424) 238-8076,7573.472066
4,LXhSp1RYSHIEZ4c01ewmPA,loqui-culver-city,Loqui,https://s3-media2.fl.yelpcdn.com/bphoto/wjV3vE...,False,https://www.yelp.com/biz/loqui-culver-city?adj...,807,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.0274352, 'longitude': -118.387...",[delivery],$$,"{'address1': '8830 Washington Blvd', 'address2...",+19499972800,(949) 997-2800,2268.934966


In [33]:
df.tail()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,4zU-v2OxDSosvNpFb37whw,the-court-cafe-los-angeles,The Court Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/1wWpEl...,False,https://www.yelp.com/biz/the-court-cafe-los-an...,552,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 33.9767818448418, 'longitude': -1...","[restaurant_reservation, delivery]",$$,"{'address1': '5496 W Centinela Ave', 'address2...",+13104314969,(310) 431-4969,5759.399143
996,qSjl8vLrZQhWDc2ofXoBwQ,golazo-restaurant-los-angeles,Golazo Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/2GLReA...,False,https://www.yelp.com/biz/golazo-restaurant-los...,9,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.0112549, 'longitude': -118.277...",[delivery],$,{'address1': '245 W Martin Luther King Jr Blvd...,+13232350102,(323) 235-0102,12305.581654
997,w0cm7DFmBDQ_x2r83cvL6g,apollonias-pizzeria-los-angeles-2,Apollonia's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/7y6Mbz...,False,https://www.yelp.com/biz/apollonias-pizzeria-l...,814,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 34.06188, 'longitude': -118.34278}",[delivery],$$,"{'address1': '5176 Wilshire Blvd', 'address2':...",+13239372823,(323) 937-2823,7787.541613
998,W35Y45Jue534WnB9NzDWEg,gutierrez-mexican-restaurant-moorpark-2,Gutierrez Mexican Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/5f6EoB...,False,https://www.yelp.com/biz/gutierrez-mexican-res...,132,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.2784839254345, 'longitude': -1...",[delivery],$,"{'address1': '304 W Los Angeles Ave', 'address...",+18055297862,(805) 529-7862,52328.241849
999,Wr1NPE3LT7Yly79gn4KlSQ,panxa-cocina-long-beach,Panxa Cocina,https://s3-media2.fl.yelpcdn.com/bphoto/JJ_bX1...,False,https://www.yelp.com/biz/panxa-cocina-long-bea...,1739,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 33.764576, 'longitude': -118.1465...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '3937 E Broadway', 'address2': ''...",+15624337999,(562) 433-7999,37457.446533


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1000 non-null   object 
 1   alias          1000 non-null   object 
 2   name           1000 non-null   object 
 3   image_url      1000 non-null   object 
 4   is_closed      1000 non-null   bool   
 5   url            1000 non-null   object 
 6   review_count   1000 non-null   int64  
 7   categories     1000 non-null   object 
 8   rating         1000 non-null   float64
 9   coordinates    1000 non-null   object 
 10  transactions   1000 non-null   object 
 11  price          800 non-null    object 
 12  location       1000 non-null   object 
 13  phone          1000 non-null   object 
 14  display_phone  1000 non-null   object 
 15  distance       1000 non-null   float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 118.3+ KB
